In [2]:
!pip install torch

     |████████████████████████████████| 80.5 MB 3.1 MB/s eta 0:00:011    |███████████▍                    | 28.6 MB 2.5 MB/s eta 0:00:21     |███████████████████████▌        | 59.0 MB 26.9 MB/s eta 0:00:01     |███████████████████████████▉    | 70.1 MB 4.0 MB/s eta 0:00:03     |██████████████████████████████  | 75.2 MB 676 kB/s eta 0:00:08     |██████████████████████████████  | 75.3 MB 676 kB/s eta 0:00:08
     |████████████████████████████████| 829 kB 1.9 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=43e11029d1100d32ce53d8a37c1fd41d9c47af3de41807a65c26b3481df1405b
  Stored in directory: /Users/andreyminin/Library/Caches/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built future
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

print(torch.__version__)

1.5.0


In [4]:
train = pd.read_csv('/content/sample_data/mnist_train_small.csv')
train.shape

FileNotFoundError: [Errno 2] File b'/content/sample_data/mnist_train_small.csv' does not exist: b'/content/sample_data/mnist_train_small.csv'

In [3]:
test = pd.read_csv('/content/sample_data/mnist_test.csv')
test.shape

(9999, 785)

In [4]:
train

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,...,0.551,0.552,0.553,0.554,0.555,0.556,0.557,0.558,0.559,0.560,0.561,0.562,0.563,0.564,0.565,0.566,0.567,0.568,0.569,0.570,0.571,0.572,0.573,0.574,0.575,0.576,0.577,0.578,0.579,0.580,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19996,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19997,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
y_train = train.iloc[:, 0]
X_train = train.iloc[:, 1:]

y_test = test.iloc[:, 0]
X_test = test.iloc[:, 1:]

n_samples, n_features = X_train.shape
X_train = X_train.astype(np.float32)
# print(str(y_train.shape))
# print(str(X_train.shape))

In [0]:
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_train = torch.tensor(X_train.values, dtype=torch.float32)

y_test = torch.tensor(y_test.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

In [0]:
weight = torch.from_numpy(np.random.normal(0, 0.1, (784, 10))).float().to('cpu')
bias = torch.from_numpy(np.random.normal(0, 0.1, 10)).float().to('cpu')

In [14]:
y_pred = X_train[:100] @ weight + bias
y_pred.shape

torch.Size([100, 10])

In [0]:
def outer(p):
  """Compute outer product pp'"""
  p = torch.unsqueeze(p, 1)
  return p @ p.T

def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

from numpy import linalg as LA
def M_count(x): 
    p = torch.argmax(torch.exp(log_softmax(x)), dim=1)
    H = torch.diag(p) - outer(p)
    w, _ = torch.eig(H)
    return torch.max(H)

In [16]:
log_softmax(y_pred).shape

torch.Size([100, 10])

In [17]:
torch.diag(y_pred)

tensor([ 233.6172,  -69.2909,  252.5618,  110.8755,  177.2188,  383.6251,
          87.8321,  152.3576, -197.6725, -397.1175])

In [18]:
outer(torch.argmax(torch.exp(y_pred), dim=1)).shape

torch.Size([100, 100])

In [19]:
torch.eig(torch.diag(torch.argmax(torch.exp(y_pred), dim=1)) - outer(torch.argmax(torch.exp(y_pred), dim=1)))

RuntimeError: ignored

In [0]:
M_count(y_pred)

In [0]:
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_train = torch.tensor(X_train.values, dtype=torch.float32)

y_test = torch.tensor(y_test.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

# X_train = torch.from_numpy(np.array(X_train).float().to(device)
# y_train = torch.from_numpy(y_train).float().to(device)
# X_test = torch.from_numpy(X_test).float().to(device)
# y_test = torch.from_numpy(y_test).float().to(device)

In [0]:
y_test = y_test.reshape(9999, 1)
y_train = y_train.reshape(19999, 1)

In [0]:
y_train

tensor([[5.],
        [7.],
        [9.],
        ...,
        [2.],
        [9.],
        [5.]])

In [0]:
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [0]:
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [0]:
batch_size = 100
n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784
output_dim = 10
lr_rate = 0.001

In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
train_loader

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


torch.Size([19999, 784])
torch.Size([19999, 1])
torch.Size([9999, 784])
torch.Size([9999, 1])


In [0]:
# class LogisticRegression(nn.Module):

#   def __init__(self, inp_features_am, oup_fetautes_am):
#     super(LogisticRegression, self).__init__()
#     self.linear = nn.Linear(inp_features_am, oup_fetautes_am)

#   def forward(self, x):
#     y_predicted = torch.sigmoid(self.linear(x))
#     return y_predicted
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [0]:
model = LogisticRegression(input_dim, output_dim)

In [0]:
model.train()
model.parameters

<bound method Module.parameters of LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)>

In [0]:
# lr = 0.01
cross_entropy_loss = nn.BCELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [0]:
import random
def f_grad(x):
  return 1 / x

def y_k_plus_1(x_k):
  return -f_grad(x_k) / M + x_k

def generateL_k(M):
  return random.random() * (4 * M) + 4 * M

In [0]:
from torch.optim.optimizer import Optimizer

In [0]:
class MyCustomOptimizer(Optimizer):
  def __init__(self, params, lr=0.01, A0=0, M=0, x=0):
    defaults = dict(lr=lr, A0=A0, M=M, x=x)
    super(MyCustomOptimizer, self).__init__(params, defaults)
  # @torch.no_grad()
  def step(self, closure=None):
    loss = None
    for group in self.param_groups:
        for p in group['params']:
            # print(p.data)
            state = self.state[p]
            if len(state) == 0:
                state['step'] = 0
                p.data = torch.from_numpy(np.random.normal(0, 0.1, p.data.shape)).float().to('cpu')
                state['u'] = torch.from_numpy(np.float64(p.data + np.random.normal(0, 0.1, p.data.shape))).float().to('cpu')
                state['A'] = group['A0']
                state['x'] = torch.from_numpy(np.random.normal(0, 0.1, p.data.shape)).float().to('cpu')
                state['M'] = np.power(p.data.min(), -2)
            p.grad = torch.div(1.0, p.data)
            grad = p.grad
            A_k = state['A']
            x = group['x']
            u = state['u']
            M = np.power(3.0, 2, dtype=np.float32)

            state['step'] += 1
            L_k = generateL_k(M)
            lr = group['lr']
            lr = float((1 / L_k + np.sqrt(1 / (L_k * L_k) + 4 * A_k / L_k)) / 2)
            x = p.data.mul_(A_k).div(A_k + lr) + u.mul_(lr).div(A_k + lr)
            # print(p.data.mul_(A_k).div(A_k + lr))
            # print(u.mul_(lr).div(A_k + lr))
            p.data = torch.add(x, -grad.div(group['M']))

            u = u - torch.div(1.0, p.data).mul(lr)
            print(u)
    return loss


In [0]:
from torch.autograd import Variable

In [0]:
criterion = torch.nn.CrossEntropyLoss()

In [0]:
iter = 0
optimizer = MyCustomOptimizer(model.parameters(), lr=lr_rate, A0=0, M=0, x=0)
# optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
for epoch in range(1000):
    model.train()
    images = X_train
    labels = y_train
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    model.float()
    optimizer.step()

    iter+=1
    # if iter >= 20:
    #   break
    if iter%200==0:
        # print(outputs)
        # calculate Accuracy
        correct = 0
        # total = 0
        # for images, labels in test_loader:
        #     images = Variable(images.view(-1, 28*28))
        #     outputs = model(images)
        #     _, predicted = torch.max(outputs.data, 1)
        #     total+= labels.size(0)
        #     # for gpu, bring the predicted and labels back to cpu fro python operations to work
        #     correct+= (predicted == labels).sum()
        # accuracy = 100 * correct/total
        print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), 0))

RuntimeError: ignored

In [0]:
iter = 0
optimizer = MyCustomOptimizer(model.parameters(), lr=lr_rate, A0=0, M=0, x=0)
# optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
for epoch in range(int(epochs)):
    if iter > 0:
      break
    for i, (images, labels) in enumerate(train_loader):
        if i > 0:
          continue
        # model.train()
        images = Variable(images.view(-1, 28 * 28))
        # print(images.shape)
        labels = Variable(labels)
        # print(images.dtype)
        # print(labels.dtype)
        optimizer.zero_grad()
        outputs = model(images)
        # print(outputs)
        loss = criterion(outputs, labels.long())
        # print(loss)
        loss.backward()
        model.float()
        optimizer.step()
        # model.float()

        iter+=1
        # if iter >= 20:
        #   break
        if iter%200==0:
            # print(outputs)
            # calculate Accuracy
            correct = 0
            # total = 0
            # for images, labels in test_loader:
            #     images = Variable(images.view(-1, 28*28))
            #     outputs = model(images)
            #     _, predicted = torch.max(outputs.data, 1)
            #     total+= labels.size(0)
            #     # for gpu, bring the predicted and labels back to cpu fro python operations to work
            #     correct+= (predicted == labels).sum()
            # accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), 0))

tensor([[ 5.3762e-03, -6.5552e-04,  8.4632e-04,  ..., -3.3123e-03,
         -2.3730e-03,  3.9919e-04],
        [-3.8291e-03, -4.2614e-03, -1.9443e-05,  ..., -1.0645e-03,
         -1.8914e-03, -4.7235e-03],
        [-7.3326e-04,  4.6878e-03, -6.4139e-04,  ..., -3.9680e-04,
          2.7606e-04, -3.3351e-04],
        ...,
        [-5.0320e-03,  6.7918e-03,  2.5459e-03,  ..., -6.8801e-03,
          7.3960e-03,  4.1206e-03],
        [ 3.0568e-03,  4.7950e-03,  3.5778e-03,  ..., -4.4115e-03,
          2.7383e-03, -3.8476e-03],
        [ 2.1404e-03, -6.2227e-03,  5.5243e-04,  ...,  2.6471e-03,
          2.9367e-04,  1.4112e-03]])
tensor([-0.0001, -0.0001,  0.0064, -0.0041,  0.0037, -0.0019,  0.0032, -0.0007,
        -0.0016,  0.0029])


In [0]:
epochs_num = 100
optimizer = MyCustomOptimizer(model.parameters(), lr=0.001, A0=0, M=0)
for epoch in range(epochs_num):
    optimizer.zero_grad()
    model.train()
    print(type(X_train))
    y_pred = model(X_train.double())
    loss = cross_entropy_loss(y_pred, y_train.double())
    loss.backward()
    optimizer.step(cross_entropy_loss)
    if (epoch + 1) % 10 == 0:
         print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')

<class 'torch.Tensor'>


RuntimeError: ignored

In [0]:
epochs_num = 100
weights = model.parameters()
u_k = weights
M = np.power(np.min(weights), -2, dtype = np.float)
A_k = 0
for epoch in range(epochs_num):
  y_pred = np.dot(weights, )
  loss = cross_entropy_loss(y_pred, y_train)
  loss.backward()
  weights = model.parameters()

  L_k = generateL_k(M)

  lr = (1 / L_k + np.sqrt(1 / (L_k * L_k) + 4 * A_k / L_k)) / 2
  A_kplus1 = A_k + lr
  x_k = A_k / A_kplus1 * weights + lr / A_kplus1 * u_k
  weights = y_k_plus_1(x_k)
  u_k = u_k - lr * f_grad(weights)
  A_k = A_kplus1

  if (epoch + 1) % 10 == 0:
    print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')


SyntaxError: ignored

In [0]:
def f(x):
  pass

In [0]:
def F_L_z(L, z, x):
  return f(x) + L / 2 * (np.linalg.norm(x - z) ** 2)

In [0]:
def count_y (f_L_k, x_k, p, M_p) :
  